*This notebook was developed by Marina Ricci for the DESC CL_Cosmo_Pipeline team.*
# This notebook aims at presenting the cluster pipeline for 
# stacked $\Delta\Sigma$ profiles and counts in richness/redshift bins.

It is meant to be run independently. If you already produced the necessary outputs `ceci` will use them, and otherwise create them.

___

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pprint import pprint

import ceci
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import Image
from astropy.table import Table

import re
import sacc

Make sure to change your path in the next cell that leads to your TXPipe directory. See examples for IN2P3 and NERSC below.

In [5]:
# user specific paths -- IN2P3 example
my_txpipe_dir = "/sps/lsst/users/ebarroso/TXPipe/"

# user specific paths -- NERSC example
# my_txpipe_dir = "/pscratch/sd/a/avestruz/TXPipe"

os.chdir(my_txpipe_dir)

import txpipe

___

In [4]:
step1 = txpipe.TXSourceSelectorMetadetect.make_stage(
    # This file is the input metadetect shear catalog
    shear_catalog=f"{my_txpipe_dir}/data/example/inputs/metadetect_shear_catalog.hdf5",
    # This is an input training set for the tomographic selection
    calibration_table=f"{my_txpipe_dir}data/example/inputs/sample_cosmodc2_w10year_errors.dat",

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/examples/metadetect/config.yml",

    # This is the output file for this stage
    output_dir=f"{my_txpipe_dir}/data/example/outputs_metadetect/"
)

In [5]:
step1.run()
step1.finalize()

Process 0 running selection for rows 0-82,200


In [6]:
step2 = txpipe.CLClusterBinningRedshiftRichness.make_stage(
    # This file is the input metadetect shear catalog
    cluster_catalog=f"{my_txpipe_dir}data/example/inputs/cluster_catalog.hdf5",
    # This is an input training set for the tomographic selection

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/notebooks/cluster_counts/gt_treecor_test/config-1deg2-CL.yml",

    # This is the output file for this stage
    output_dir=f"{my_txpipe_dir}/data/example/output_binning/"
)



In [7]:
step2.run()
step2.finalize()

In [8]:
step3 = txpipe.extensions.cluster_counts.TXTwoPointCluster.make_stage(
    # This file is the input metadetect shear catalog
    binned_lens_catalog=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    # This is an input training set for the tomographic selection
    binned_shear_catalog=f"{my_txpipe_dir}shear_tomography_catalog.hdf5",
    binned_random_catalog=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    patch_centers=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    tracer_metadata=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    config=f"{my_txpipe_dir}notebooks/cluster_counts/gt_treecor_test/config-1deg2-CL.yml",
    output_dir=f"{my_txpipe_dir}data/example/output_binning/"
)

In [9]:
step3.run()
step3.finalize()

KeyError: "Unable to synchronously open object (object 'shear' doesn't exist)"

In [ ]:
step4 = txpipe.TXShearCalibration.make_stage(
    # This file is the input metadetect shear catalog
    shear_catalog=f"{my_txpipe_dir}data/example/inputs/shear_catalog.hdf5",
    shear_tomography_catalog=f"{my_txpipe_dir}shear_tomography_catalog.hdf5",
    # This is an input training set for the tomographic selection

    # This contains all the options for this stage. You can override them here
    # manually too.
    # This is the output file for this stage
    output_dir=f"{my_txpipe_dir}/data/example/output_binning_shear/"
)



In [ ]:
step4.run()
step4.finalize()

## Tests

In [ ]:
import h5py

In [ ]:
with h5py.File(f"{my_txpipe_dir}shear_tomography_catalog.hdf5", "r") as data:
    print(data.keys())
    for key in data.keys():
        print(key, data[key].attrs.keys())

In [ ]:
tb = h5py.File(f"{my_txpipe_dir}/shear_tomography_catalog.hdf5", "r")

In [ ]:
for k in tb["counts"].keys():
    print(k, tb["counts"][k])

In [ ]:
tb["tomography"]["bin"][:]

In [ ]:
data = h5py.File(f"{my_txpipe_dir}/data/example/inputs/metadetect_shear_catalog.hdf5", "r")

In [ ]:
Table(data["shear"]["00"])